In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import seaborn as sns


sns.set(style="whitegrid")
palette = sns.color_palette("husl", 8)
ITALIAN_COLOR = sns.color_palette("husl", 8)[0]
ENGLISH_COLOR = sns.color_palette("husl", 8)[3]

## Imports

In [ ]:
!pip install datasets

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.9 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=a3034f284fedd1e1fcec122abae061911614264ded9394b77e738bde8d589178
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [ ]:
!python -m spacy download it_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 68.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 86.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 51.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import pandas as pd
import spacy
import random
from nltk.corpus import stopwords
import re
import nltk
import numpy as np
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from tqdm import tqdm

## Verbs lists

### Italian

In [ ]:
italian_verbs = ['toccare', 'sfiorare', 'colpire', 'urtare', 'sbattere', 'tamponare',
                'scontrarsi', 'spingere', 'spintonare', 'schiaffeggiare', 'calciare',
                'leccare', 'baciare', 'frustare', 'punzecchiare', 'pungere', 'schiacciare',
                'premere', 'cliccare', 'appoggiare', 'poggiare'
                 ]

In [ ]:
italian_to_russian = {
    "toccare": "тронуть",
    "sfiorare": "коснуться",
    "colpire": "ударить",
    "urtare": "удариться",
    "sbattere": "удариться",
    "tamponare": "врезаться",
    "scontrarsi": "столкнуться",
    "spingere": "толкнуть",
    "spintonare": "толкнуть",
    "schiaffeggiare": "шлёпнуть",
    "calciare": "пнуть",
    "leccare": "лизнуть",
    "baciare": "поцеловать",
    "frustare": "хлестнуть",
    "punzecchiare": "кольнуть",
    "pungere": "кольнуть",
    "schiacciare": "нажать",
    "premere": "нажать",
    "cliccare": "кликнуть",
    "appoggiare": "приложить",
    "poggiare": "приложить"
}

### English

In [ ]:
english_verbs = ['touch', 'hit', 'strike', 'beat', 'bang', 'bash', 'punch', 'whack',
                 'kick', 'smack', 'slap', 'swat', 'pat', 'tap', 'push', 'shove',
                 'crash', 'bump', 'collide', 'graze', 'nick'
                 ]

In [ ]:
english_to_russian = {
    "touch": "тронуть",
    "hit": "ударить",
    "strike": "ударить",
    "beat": "ударить",
    "bang": "ударить",
    "bash": "разбить",
    "punch": "ударить",
    "whack": "шлёпнуть",
    "kick": "пнуть",
    "smack": "шлёпнуть",
    "slap": "пощечина",
    "swat": "поймать",
    "pat": "погладить",
    "tap": "постучать",
    "push": "толкнуть",
    "shove": "толкнуть",
    "crash": "разбиться",
    "bump": "столкнуться",
    "collide": "столкнуться",
    "graze": "коснуться",
    "nick": "царапнуть"
}

## Italian Tweets 10M

### Скачиваем данные

https://huggingface.co/datasets/pere/italian_tweets_10M

In [ ]:
italian_data = load_dataset("pere/italian_tweets_10M")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/228 [00:00<?, ?B/s]

italian_tweets_10M.py:   0%|          | 0.00/2.74k [00:00<?, ?B/s]

The repository for pere/italian_tweets_10M contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/pere/italian_tweets_10M.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Repo card metadata block was not found. Setting CardData to empty.


train-shard-0001-of-0001.json.gz:   0%|          | 0.00/515M [00:00<?, ?B/s]

validation-shard-0001-of-0001.json.gz:   0%|          | 0.00/27.1M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
italian_data

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 9500000
    })
    validation: Dataset({
        features: ['id', 'text'],
        num_rows: 500000
    })
})

### Обрабатываем строки

In [ ]:
class ItalianTextProcessor:
    def __init__(self, italian_sample, italian_verbs, italian_to_russian, output_path):
        self.italian_sample = italian_sample
        self.italian_verbs = italian_verbs
        self.italian_to_russian = italian_to_russian
        self.output_path = output_path
        self.nlp = spacy.load('it_core_news_sm')
        nltk.download('stopwords')
        self.stop_words = set(stopwords.words('italian'))

    def lemmatize_and_filter(self, text):
        """Lemmatize text and check for the presence of specific verbs."""
        doc = self.nlp(text)
        lemmas = {token.lemma_ for token in doc if token.pos_ == 'VERB'}
        found_verbs = lemmas & self.italian_verbs_set
        return len(found_verbs) > 0, list(found_verbs)

    def process_italian_data(self):
        """Process Italian data to filter rows with specific verbs and preprocess text."""
        self.italian_verbs_set = set(self.italian_verbs)

        tqdm.pandas(desc="Filtering rows")
        italian_dataset = self.italian_sample.to_pandas()
        italian_dataset['has_verb'], italian_dataset['found_verbs'] = zip(*italian_dataset['text'].progress_apply(lambda x: self.lemmatize_and_filter(x)))

        filtered_italian_df = italian_dataset[italian_dataset['has_verb']]
        filtered_italian_df = filtered_italian_df.drop(columns=['has_verb'])

        filtered_italian_df['found_verbs'] = filtered_italian_df['found_verbs'].apply(lambda x: list(set(x)))
        filtered_italian_df = filtered_italian_df[filtered_italian_df['found_verbs'].apply(lambda x: len(x) == 1)]
        filtered_italian_df['found_verbs'] = filtered_italian_df['found_verbs'].apply(lambda x: x[0])

        filtered_italian_df['found_verbs'] = filtered_italian_df['found_verbs'].apply(lambda verb: f'{verb} ({self.italian_to_russian.get(verb, "Перевод не найден")})')

        filtered_italian_df.to_csv(self.output_path)

        return filtered_italian_df, len(filtered_italian_df)

In [ ]:
# Пример использования
italian_sample = italian_data['train'].select(range(500000))
output_path = "/content/drive/MyDrive/Диплом бакалавриат'25/Data/italian_500000.csv"

processor = ItalianTextProcessor(italian_sample, italian_verbs, italian_to_russian, output_path)
italian_df, length = processor.process_italian_data()
print(f"Processed {length} rows.")

### Объединяем в единый файл

In [ ]:
it_1 = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/Data/italian_500000.csv")
it_2 = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/Data/italian_500_1500.csv")
it_3 = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/Data/italian_1500_2500.csv")
it_4 = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/Data/italian_2500_3500.csv")
it_5 = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/Data/italian_3500_4500.csv")
it_6 = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/Data/italian_4500_5500.csv")
it_7 = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/Data/italian_5500_6500.csv")
it_8 = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/Data/italian_6500_7500.csv")
it_9 = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/Data/italian_7500_8500.csv")
it_10 = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/Data/italian_8500_9500.csv")
it_11 = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/Data/italian_val_500.csv")

In [ ]:
combined_df = pd.concat([it_1, it_2, it_3, it_4, it_5, it_6, it_7, it_8, it_9, it_10, it_11], ignore_index=True)

In [ ]:
combined_df = combined_df[['id', 'text', 'found_verbs']]

In [ ]:
combined_df.sample(5)

,id,text,found_verbs
36904,4551620,preoccupante aumento dei ricoveri tra i pienam...,spingere (толкнуть)
57497,7076519,@user: sanremo. 18enne soccorritore colpito da...,colpire (ударить)
11468,1419848,"@user: più opprimono spingendo al #vaccino, pi...",spingere (толкнуть)
11371,1408662,@user: la svezia non ha l'euro quindi economic...,sbattere (удариться)
76685,9399595,"@user: ecco qua… “in israele, i ricercatori ha...",colpire (ударить)


In [ ]:
combined_df.to_csv("/content/drive/MyDrive/Диплом бакалавриат'25/final_italian_data.csv", index=False)

## English Twitter 100M tweets

### Скачиваем данные

In [ ]:
english_data = load_dataset("enryu43/twitter100m_tweets")

NameError: name 'load_dataset' is not defined

In [ ]:
english_data

DatasetDict({
    train: Dataset({
        features: ['user', 'id', 'tweet', 'replies', 'retweets', 'likes', 'quotes', 'date'],
        num_rows: 88084332
    })
})

### Обрабатываем строки

In [ ]:
class TweetProcessor:
    def __init__(self, english_data, english_verbs, english_to_russian):
        self.nlp = spacy.load('en_core_web_sm')
        nltk.download('stopwords')
        self.stop_words = set(stopwords.words('english'))
        self.english_data = english_data
        self.english_verbs = english_verbs
        self.english_to_russian = english_to_russian

    def lemmatize_and_filter(self, text):
        """Lemmatize text and check for the presence of specific verbs."""
        doc = self.nlp(text)
        lemmas = {token.lemma_ for token in doc if token.pos_ == 'VERB'}
        found_verbs = lemmas & set(self.english_verbs)
        return len(found_verbs) > 0, list(found_verbs)

    def process_english_data_chunk(self, english_sample, output_path):
        """Process a chunk of English data to filter rows with specific verbs and preprocess text."""
        english_verbs_set = set(self.english_verbs)

        tqdm.pandas(desc="Filtering rows")
        english_dataset = english_sample.to_pandas()
        english_dataset['has_verb'], english_dataset['found_verbs'] = zip(*english_dataset['tweet'].progress_apply(lambda x: self.lemmatize_and_filter(x)))

        filtered_english_df = english_dataset[english_dataset['has_verb']]
        filtered_english_df = filtered_english_df.drop(columns=['has_verb'])

        filtered_english_df['found_verbs'] = filtered_english_df['found_verbs'].apply(lambda x: list(set(x)))
        filtered_english_df = filtered_english_df[filtered_english_df['found_verbs'].apply(lambda x: len(x) == 1)]
        filtered_english_df['found_verbs'] = filtered_english_df['found_verbs'].apply(lambda x: x[0])

        filtered_english_df['found_verbs'] = filtered_english_df['found_verbs'].apply(lambda verb: f'{verb} ({self.english_to_russian.get(verb, "Перевод не найден")})')

        filtered_english_df.to_csv(output_path)

        return filtered_english_df.head(), len(filtered_english_df)

    def process_single_chunk(self, chunk_index, output_dir):
        """Process a single chunk of 1 million rows from a larger subset of 10 million rows."""
        chunk_size = 10_000_000
        sample_size = 1_000_000
        start_index = chunk_index * chunk_size

        chunk = self.english_data['train'].select(range(start_index, start_index + chunk_size))
        random_sample = chunk.select(random.sample(range(len(chunk)), sample_size))

        output_path = f"{output_dir}/english_chunk_{chunk_index + 1}.csv"
        self.process_english_data_chunk(random_sample, output_path)

    def process_remaining_data(self, chunk_index, output_dir):
        """Process the remaining data after the first 8 chunks."""
        sample_size = 2_000_000
        start_index = chunk_index * 10_000_000

        chunk = self.english_data['train'].select(range(start_index, len(self.english_data['train'])))
        random_sample = chunk.select(random.sample(range(len(chunk)), sample_size))

        output_path = f"{output_dir}/english_chunk_{chunk_index + 1}.csv"
        self.process_english_data_chunk(random_sample, output_path)

In [ ]:
processor = TweetProcessor(english_data, english_verbs, english_to_russian)

In [ ]:
processor.process_single_chunk(chunk_index=0, output_dir="/content/drive/MyDrive/Диплом бакалавриат'25")

Preprocessing...: 100%|██████████| 7350/7350 [01:27<00:00, 84.22it/s]


In [ ]:
processor.process_single_chunk(chunk_index=1, output_dir="/content/drive/MyDrive/Диплом бакалавриат'25")

Preprocessing...: 100%|██████████| 7593/7593 [01:33<00:00, 80.98it/s]


In [ ]:
processor.process_single_chunk(chunk_index=2, output_dir="/content/drive/MyDrive/Диплом бакалавриат'25")

Preprocessing...: 100%|██████████| 7456/7456 [01:29<00:00, 83.63it/s]


In [ ]:
processor.process_single_chunk(chunk_index=3, output_dir="/content/drive/MyDrive/Диплом бакалавриат'25")

Preprocessing...: 100%|██████████| 7525/7525 [01:28<00:00, 85.47it/s]


In [ ]:
processor.process_single_chunk(chunk_index=4, output_dir="/content/drive/MyDrive/Диплом бакалавриат'25")

Preprocessing...: 100%|██████████| 7578/7578 [01:29<00:00, 84.61it/s]


In [ ]:
processor.process_single_chunk(chunk_index=5, output_dir="/content/drive/MyDrive/Диплом бакалавриат'25")

Preprocessing...: 100%|██████████| 7497/7497 [01:34<00:00, 79.32it/s]


In [ ]:
processor.process_single_chunk(chunk_index=6, output_dir="/content/drive/MyDrive/Диплом бакалавриат'25")

Preprocessing...: 100%|██████████| 7263/7263 [01:34<00:00, 76.78it/s]


In [ ]:
processor.process_single_chunk(chunk_index=7, output_dir="/content/drive/MyDrive/Диплом бакалавриат'25")

Preprocessing...: 100%|██████████| 7503/7503 [01:30<00:00, 83.07it/s]


Т.к. в данных 88 миллионов строк, то немного изменим код далее и сразу обработаем 2 миллиона строк.

In [ ]:
processor.process_remaining_data(chunk_index=8, output_dir="/content/drive/MyDrive/Диплом бакалавриат'25")

Preprocessing...: 100%|██████████| 15088/15088 [03:08<00:00, 79.90it/s]


### Объединяем в единый файл

In [ ]:
file_names = [f"/content/drive/MyDrive/Диплом бакалавриат'25/Data/english_chunk_{i}.csv" for i in range(1, 10)]

df_list = [pd.read_csv(file) for file in file_names]

In [ ]:
combined_df = pd.concat(df_list, ignore_index=True)
combined_df = combined_df[['user', 'id', 'tweet', 'date', 'found_verbs']]

In [ ]:
combined_df.sample(5)

,user,id,tweet,date,found_verbs
63223,EdgarGodineaux,1316771503543738369,If you’ve been pushing your health goals to th...,2020-10-15T16:02:45+00:00,push (толкнуть)
21234,noellejoyeuse,1054793997129080833,@RightWingLawMan That's a white supremacist ge...,2018-10-23T17:57:40+00:00,punch (ударить)
23721,Networkinvegas,1548877880901242881,@reviewjournal Local media is pushing this cau...,2022-07-18T03:50:38+00:00,push (толкнуть)
65306,HexisConsulting,1536409521140154370,Ransomware is one of the most devastating type...,2022-06-13T18:05:49+00:00,hit (ударить)
23528,Wrestle40,1543615245452775426,"📻 After 18 weeks on the charts, @HumanLeagueHQ...",2022-07-03T15:18:48+00:00,hit (ударить)


In [ ]:
combined_df.to_csv("/content/drive/MyDrive/Диплом бакалавриат'25/final_english_data.csv", index=False)